In [ ]:
''' FOR DEVELOPMENT ONLY, DELETE CELL '''

# set auto reload imported modules tagged
%load_ext autoreload
%autoreload 2

# data array processing
#import numpy as np
#import pandas as pd

# plotting with matplotlib
#%matplotlib widget
#from matplotlib import pyplot as plt

## development override params
# pvlibs_path = '/home/brendan/dev/pvlibs'
# base_path = '../data/slt'


In [ ]:
''' initialise notebook '''

# add location to path
pvlibs_path = '/srv/pvlibs'
import sys
sys.path.append(pvlibs_path)

# plotting with matplotlib, set interactive figure display
%matplotlib widget

# import pvlibs package
import pvlibs


In [ ]:
''' import measurement files '''

# define directory to search; "./" for current directory, "../" for up one directory
base_path = '../data/pl/'

# define measurement properties (!required!)
props = {
    # measurement type
    'meas_type': 'ocpl', # 'ocpl' for open-circuit photoluminescence
    'file_ext': 'tif', # ['tif']
    'file_type': 'tif', # ['tif']
    
    # wafer properties
    
    # measurement conditions
    'exposure': 0.5, # define global pl exposure, read from file if available

}

# generate measurement file database
db = pvlibs.init_file_db(base_path, props)


In [ ]:
''' parse parameters from file name '''

# example: "h555_fired-830C_D23_TC.ltr"

# define single parameter separator; e.g. '-', '_', or '\s' (space)
param_sep = '\s'

# define ordered list of parameters within filename separated by param_sep
params = [
    'plid',
    'misc1',
    'device',
    'misc2',
    'misc3',
    'misc4',
]

# parse file name parameters from each measurement file
db = pvlibs.parse_file_names(db, param_sep, params)


In [ ]:
''' import image data '''

# import data from files
db = pvlibs.import_file_data(db)


In [ ]:
''' normalise image exposure '''

# define reference exposure to normalise (or leave none for auto maximum value)
ref_exp = None

# normalise pl images to reference exposure
db = pvlibs.norm_pl_exposure(db, ref_exp)


In [ ]:
''' histogram and stats of image data '''

# define histogram config
params = {
    'bins': 100, # number of histogram bins
    'floor': 3500, # noise floor to remove in pl cnts
}

# calculate pl image count statistics and histogram
db = pvlibs.pl_hist_stats(db, params)


In [ ]:
''' plot histogram with stats '''

import numpy as np
import matplotlib.pyplot as plt


# select image node
node = db[8]


# diplay images
_w = 7; _h = 5; fig = plt.figure(figsize = (_w, _h))
fig.canvas.layout.width = '{}in'.format(_w); fig.canvas.layout.height= '{}in'.format(_h)
#plt.xticks([]); plt.yticks([])

plt.xlabel('Photoluminescence Intensity (Cnts.)')
plt.ylabel('Area Fraction')


x = node['hist_bins']
hist = node['hist_norm']
#hist = node['hist_cnts']

# stem plot histogram
st = plt.stem(x, hist, linefmt = '-', markerfmt = '-', basefmt = 'k-', use_line_collection = True)

m = node['med']
plt.vlines(m, 0., np.max(hist)*1.1, colors = 'r')

s = node['std']
plt.vlines([m-s, m+s], 0., np.max(hist)*1.1, colors = 'k', linestyles = '--', alpha = 0.5)


# display figure
plt.tight_layout()
plt.show()


In [ ]:
''' save exposure normalised images '''
    
# set output file path and header 
file_name_head = './normpl'

# define ordered list of parameters within filename separated by param_sep
params = [
    'plid',
    'device',
]

# save normalised pl images to file [tif]
pvlibs.save_norm_pl(db, file_name_head, params)


In [ ]:
''' plot and save all histograms '''

# set output file path and header 
file_name_head = './pl-hist'

# define ordered list of parameters within filename separated by param_sep
params = [
    'plid',
    'device',
]

# prepare histogram plot and save for all images
pvlibs.save_pl_hist(db, file_name_head, params)


In [ ]:
''' compile and export data'''

# define list of measurement labels to export; format each as 'file_name_param': 'Output Label'
labels = {
    'plid': 'ID',
    'device': 'Device',
}

# select values to export; format each as 'param': 'Output Label'
values = {
    'exposure': 'Exposure [s]',
    'intensity': 'Light Intensity [suns]',
    'norm_exposure': 'Norm. Exposure [s]',
    'med': 'Median [cnts.]',
    'avg': 'Mean [cnts.]',
    'std': 'Std. Dev. [cnts.]',
}

# define results output file name
file_name = './pl-stats'

# compile dataset by selected labels and values
data = pvlibs.compile_data(db, labels, values, file_name)


In [ ]:
''' save all data '''

# define data for output as dict of file: data params [list]
outputs = {
    'hist': {
        'hist_bins': 'Histogram Bins [cnts.]',
        'hist_cnts': 'Histogram [pxls.]',
        'hist_norm': 'Area Norm. Histogram [pxl frac.]',
    },
}

# define ordered list of parameters to use for data output file name
params = [
    'plid',
    'device',
]

# define output file name header
file_name_head = './pl-hist'

# prepare and save all measurement and calculated data to file
pvlibs.save_all_data(db, file_name_head, params, outputs)
